<a href="https://colab.research.google.com/github/sergekamanzi/Formative2-Data-Preprocessing/blob/main/Formative_2_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Part 1: Data Augmentation on CSV Files**

In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, PowerTransformer
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Loading the dataset
data1 = pd.read_csv('/content/customer_transactions.csv', low_memory=False)
data1.head()

,customer_id_legacy,transaction_id,purchase_amount,purchase_date,product_category,customer_rating
0,151,1001,408,2024-01-01,Sports,2.3
1,192,1002,332,2024-01-02,Electronics,4.2
2,114,1003,442,2024-01-03,Electronics,2.1
3,171,1004,256,2024-01-04,Clothing,2.8
4,160,1005,64,2024-01-05,Clothing,1.3


In [4]:
data1.shape

(150, 6)

In [5]:
# Cleaning column names
data1.columns = data1.columns.str.strip()

In [6]:
# Renames purchase_amount to TransactionAmount for better readability
column_mapping = {
    'purchase_amount': 'TransactionAmount'
}
data1.rename(columns=column_mapping, inplace=True)

In [7]:
# Ensuring 'TransactionAmount' column exists
if 'TransactionAmount' not in data1.columns:
    print("Column 'TransactionAmount' not found. Available columns:", data1.columns)
    raise KeyError("TransactionAmount column missing from dataset")

In [8]:
# Handling missing values
for column in data1.select_dtypes(include=['number']).columns:
    if data1[column].isnull().sum() > 0:
        median_imputer = SimpleImputer(strategy='median')
        data1[column] = median_imputer.fit_transform(data1[[column]])

In [9]:
%%capture
data1.isnull().sum()

In [10]:
# anonymizing transaction amounts while keeping them realistic
def add_noise(series, noise_level=0.01):
    return series + noise_level * np.random.randn(len(series)) * series.std()

data1['TransactionAmount'] = add_noise(data1['TransactionAmount'])

In [11]:
# Applying Log Transformation for Skewed Data
skewed_columns = ['TransactionAmount']
for col in skewed_columns:
    data1[col] = np.log1p(data1[col])

In [12]:
data1.head()

,customer_id_legacy,transaction_id,TransactionAmount,purchase_date,product_category,customer_rating
0,151,1001,6.009810,2024-01-01,Sports,2.3
1,192,1002,5.815617,2024-01-02,Electronics,4.2
2,114,1003,6.096686,2024-01-03,Electronics,2.1
3,171,1004,5.551627,2024-01-04,Clothing,2.8
4,160,1005,4.189884,2024-01-05,Clothing,1.3


In [13]:
# Generating Synthetic Data/duplicated data
def generate_synthetic_data(df, num_samples=1000):
    synthetic_data = df.sample(n=num_samples, replace=True, random_state=42).copy()
    synthetic_data['TransactionAmount'] = add_noise(synthetic_data['TransactionAmount'], noise_level=0.02)
    synthetic_data['transaction_id'] = synthetic_data['transaction_id'].astype(str) + '_synthetic'
    return synthetic_data

synthetic_transactions = generate_synthetic_data(data1, num_samples=500)
data1 = pd.concat([data1, synthetic_transactions], ignore_index=True)

In [14]:
data1.shape

(650, 6)

In [15]:
# Saving the Processed Data
data1.to_csv('/content/customer_transactions_augmented.csv', index=False)

**Part 2: Merging Datasets with Transitive Properties**

In [27]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.display import display

In [18]:
transactions = pd.read_csv('/content/customer_transactions_augmented.csv', low_memory=False)
social_profiles = pd.read_csv('/content/customer_social_profiles.csv', low_memory=False)
id_mapping = pd.read_csv('/content/id_mapping.csv', low_memory=False)

# Display all tables
display(transactions.head(), social_profiles .head(),id_mapping.head())

,customer_id_legacy,transaction_id,TransactionAmount,purchase_date,product_category,customer_rating
0,151,1001,6.009810,2024-01-01,Sports,2.3
1,192,1002,5.815617,2024-01-02,Electronics,4.2
2,114,1003,6.096686,2024-01-03,Electronics,2.1
3,171,1004,5.551627,2024-01-04,Clothing,2.8
4,160,1005,4.189884,2024-01-05,Clothing,1.3


,customer_id_new,social_media_platform,engagement_score,purchase_interest_score,review_sentiment
0,A178,LinkedIn,74,4.9,Positive
1,A190,Twitter,82,4.8,Neutral
2,A150,Facebook,96,1.6,Positive
3,A162,Twitter,89,2.6,Positive
4,A197,Twitter,92,2.3,Neutral


,customer_id_legacy,customer_id_new
0,195,A105
1,161,A118
2,192,A156
3,157,A168
4,166,A102


In [19]:
#Removes any unwanted spaces from column names for consistency
transactions.columns = transactions.columns.str.strip()
social_profiles.columns = social_profiles.columns.str.strip()
id_mapping.columns = id_mapping.columns.str.strip()

In [21]:
#Merging Datasets

#Merge transactions with ID mapping
merged_transactions = transactions.merge(id_mapping, on='customer_id_legacy', how='left')

#Merge with social profiles
final_data = merged_transactions.merge(social_profiles, on='customer_id_new', how='left')

In [22]:
#Creating a Customer Engagement Score
final_data['CustomerEngagementScore'] = (
    final_data['engagement_score'] * 0.6 +
    final_data['purchase_interest_score'] * 0.4
)

In [23]:
#Calculating Moving Averages of Transactions
final_data['TransactionAmount_MA'] = final_data.groupby('customer_id_legacy')['TransactionAmount'].transform(lambda x: x.rolling(3, min_periods=1).mean())

In [24]:
#Time-based Aggregation of Purchases
final_data['purchase_date'] = pd.to_datetime(final_data['purchase_date'])
final_data['Year'] = final_data['purchase_date'].dt.year
final_data['Month'] = final_data['purchase_date'].dt.month

In [29]:
#Groups transactions monthly and calculates the total amount spent per customer
purchase_agg = final_data.groupby(['customer_id_legacy', 'Year', 'Month'])['TransactionAmount'].sum().reset_index()
purchase_agg.rename(columns={'TransactionAmount': 'MonthlyTotalSpent'}, inplace=True)
final_data = final_data.merge(purchase_agg, on=['customer_id_legacy', 'Year', 'Month'], how='left')

In [34]:
# Handle missing values by filling NaN with an empty string
final_data['review_sentiment'] = final_data['review_sentiment'].fillna('')

# Applies TF-IDF vectorization to convert text reviews into numeric values
if 'review_sentiment' in final_data.columns:
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(final_data['review_sentiment'])
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

    # Concatenate the TF-IDF features with the original dataframe
    final_data = pd.concat([final_data, tfidf_df], axis=1)

In [36]:
# Check if the required columns are present before scaling
columns_to_scale = ['CustomerEngagementScore', 'TransactionAmount_MA', 'MonthlyTotalSpent']

# Filter the columns that exist in final_data
existing_columns = [col for col in columns_to_scale if col in final_data.columns]

# Apply scaling only to the existing columns
scaler = MinMaxScaler()
final_data[existing_columns] = scaler.fit_transform(final_data[existing_columns])


In [38]:
# Handling Missing Values
final_data.fillna({
    'social_media_platform': 'Unknown',
    'engagement_score': 0,
    'purchase_interest_score': 0,
    'review_sentiment': 'Neutral'
}, inplace=True)


In [39]:
final_data.isnull().sum()

,0
customer_id_legacy,0
transaction_id,0
TransactionAmount,0
purchase_date,0
product_category,0
customer_rating,0
customer_id_new,133
social_media_platform,0
engagement_score,0
purchase_interest_score,0


In [ ]:
# Exporting the Final Dataset
final_data.to_csv('/content/final_customer_data_5.csv', index=False)

In [40]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.display import display

In [41]:
transactions = pd.read_csv('/content/customer_transactions_augmented.csv', low_memory=False)
social_profiles = pd.read_csv('/content/customer_social_profiles.csv', low_memory=False)
id_mapping = pd.read_csv('/content/id_mapping.csv', low_memory=False)

# Display all tables
display(transactions.head(), social_profiles .head(),id_mapping.head())

,customer_id_legacy,transaction_id,TransactionAmount,purchase_date,product_category,customer_rating
0,151,1001,6.009810,2024-01-01,Sports,2.3
1,192,1002,5.815617,2024-01-02,Electronics,4.2
2,114,1003,6.096686,2024-01-03,Electronics,2.1
3,171,1004,5.551627,2024-01-04,Clothing,2.8
4,160,1005,4.189884,2024-01-05,Clothing,1.3


,customer_id_new,social_media_platform,engagement_score,purchase_interest_score,review_sentiment
0,A178,LinkedIn,74,4.9,Positive
1,A190,Twitter,82,4.8,Neutral
2,A150,Facebook,96,1.6,Positive
3,A162,Twitter,89,2.6,Positive
4,A197,Twitter,92,2.3,Neutral


,customer_id_legacy,customer_id_new
0,195,A105
1,161,A118
2,192,A156
3,157,A168
4,166,A102


In [42]:
#Removes any unwanted spaces from column names for consistency
transactions.columns = transactions.columns.str.strip()
social_profiles.columns = social_profiles.columns.str.strip()
id_mapping.columns = id_mapping.columns.str.strip()

In [43]:
#Merging Datasets with Transitive Mapping

#Merge transactions with ID mapping
merged_transactions = transactions.merge(id_mapping, on='customer_id_legacy', how='left')

#Merge with social profiles
final_data = merged_transactions.merge(social_profiles, on='customer_id_new', how='left')

In [44]:
#Creating a Customer Engagement Score
final_data['CustomerEngagementScore'] = (
    final_data['engagement_score'] * 0.6 +
    final_data['purchase_interest_score'] * 0.4
)

In [45]:
#Calculating Moving Averages of Transactions
final_data['TransactionAmount_MA'] = final_data.groupby('customer_id_legacy')['TransactionAmount'].transform(lambda x: x.rolling(3, min_periods=1).mean())

In [46]:
#Time-based Aggregation of Purchases
final_data['purchase_date'] = pd.to_datetime(final_data['purchase_date'])
final_data['Year'] = final_data['purchase_date'].dt.year
final_data['Month'] = final_data['purchase_date'].dt.month

In [47]:
#Groups transactions monthly and calculates the total amount spent per customer
purchase_agg = final_data.groupby(['customer_id_legacy', 'Year', 'Month'])['TransactionAmount'].sum().reset_index()
purchase_agg.rename(columns={'TransactionAmount': 'MonthlyTotalSpent'}, inplace=True)
final_data = final_data.merge(purchase_agg, on=['customer_id_legacy', 'Year', 'Month'], how='left')

In [49]:
#Applies TF-IDF vectorization to convert text reviews into numeric values
final_data['review_sentiment'] = final_data['review_sentiment'].fillna('')

if 'review_sentiment' in final_data.columns:
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(final_data['review_sentiment'])
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
    final_data = pd.concat([final_data, tfidf_df], axis=1)

In [50]:
#Normalization ensures all values are scaled between 0 and 1
scaler = MinMaxScaler()
final_data[['CustomerEngagementScore', 'TransactionAmount_MA', 'MonthlyTotalSpent']] = scaler.fit_transform(
    final_data[['CustomerEngagementScore', 'TransactionAmount_MA', 'MonthlyTotalSpent']]
)

In [53]:
# Handling Missing Values
final_data.fillna({
    'social_media_platform': 'Unknown',
    'engagement_score': 0,
    'purchase_interest_score': 0,
    'review_sentiment': 'Neutral'
}, inplace=True)

In [54]:
%%capture
final_data.isnull().sum()

In [55]:
# Exporting the Final Dataset
final_data.to_csv('/content/final_customer_data_5.csv', index=False)

**Part 3: Data Consistency and Quality Checks**